In [1]:
# imports
from sympy import diff, symbols, exp, Eq, solve, lambdify
from sympy.abc import t, s
import sympy

import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import fsolve
from dataclasses import dataclass, field

import control
import json

# State Space Model:

$$\frac{dC_A}{dt}=\frac{F}{V}\left(C_{A0}-C_A\right)-k_1(T)C_A\left(C_A+C_{B0}-C_{A0}\right)$$

$$\frac{dC_S}{dt}=\frac{F}{V}\left(C_{S0}-C_S\right)-k_2(T)C_S$$

$$\frac{dT}{dt}=\frac{F}{V}\left(T_0-T\right)+\frac{-\Delta H_1k_1(T)C_A\left(C_A+C_{B0}-C_{A0}\right)-\Delta H_2k_2(T)C_S}{\rho C_p}-\frac{UA\left(T-T_c\right)}{V\rho C_p}$$

## System Parameters

In [2]:
# params
k_10 = 4e14
E_1 = 1.28e5  
k_20 = 1e84
E_2 = 8e5
H_1 = -45400
H_2 = -3.2e5
rho = 12.4
C_p = 254
A = 5.3
T_c = 373
V = 4000

F_A0 = 1075
F_S0 = 750
F_B0 = 1275

F = F_A0 + F_B0 + F_S0

v_0 = F / rho
C_A0 = F_A0 / v_0
C_B0 = F_B0 / v_0
C_S0 = F_S0 / v_0

U_s = 14.656e3
T_0s = 410

R = 8.314

## Symbolic Equation definitions

In [3]:
# define symbolic state space equations
C_A, C_A, C_S, T, T_0, U = symbols('C_A, C_A, C_S, T, T_0, U')

k_1 = k_10 * exp(-E_1 / T / R)
k_2 = k_20 * exp(-E_2 / T / R)

f_C_A = v_0 / V * (C_A0 - C_A) - k_1 * C_A * (C_A + C_B0 - C_A0)

f_C_S = v_0 / V * (C_S0 - C_S) - k_2 * C_S

f_T = v_0 / V * (T_0 - T) + (-H_1 * k_1 * C_A * (C_A + C_B0 - C_A0) - H_2 * k_2 * C_S) / (rho * C_p) - (U * A * (T - T_c)) / (V * rho * C_p)

# Steady State Evaluation

In [4]:
# functions
def solve_C_A(_T):
    _f = lambdify(C_A, f_C_A.subs({T: _T}), 'numpy')
    _C_A = fsolve(_f, [4])

    return _C_A[0]

def solve_C_S(_T):
    _f = lambdify(C_S, f_C_S.subs({T: _T}), 'numpy')
    _C_S = fsolve(_f, [4])

    return _C_S[0]

def solve_heat(_T):
    _C_A = solve_C_A(_T)
    _C_S = solve_C_S(_T)
    __f = f_T.subs({C_A: _C_A, C_S: _C_S, T_0: T_0s, U: U_s})
    _f = lambdify(T, __f, 'numpy')
    return _f(_T)

def root_solve(f, x0):
    x_n = x0
    for _ in range(100):
        dx = 1e-12
        diff = (f(x_n + dx) - f(x_n)) / dx
        x_n = x_n - f(x_n) / diff

        if np.abs(diff) < 1e-10:
            break

    return x_n

In [5]:
# T_s evaluation
T_s = root_solve(solve_heat, 460)

T_s

460.3182315699554

## Steady State Concentrations

In [6]:
# steady state outlet
C_As = solve_C_A(T_s)
C_Ss = solve_C_S(T_s)
C_As, C_Ss

(0.2116701379629191, 2.9999920948240835)

## Steady state conversion

In [7]:
# A conversion
1 - C_As / C_A0

0.9507743865202514

In [8]:
# S conversion
1 - C_Ss / C_S0

2.6350586388446118e-06

# Linearization

In [9]:
# linear matrices
A = sympy.matrices.Matrix([
    [
        f_C_A.diff(C_A).subs({C_A: C_As, T: T_s}), 
        f_C_A.diff(C_S).subs({C_A: C_As, T: T_s}), 
        f_C_A.diff(T).subs({C_A: C_As, T: T_s}),
    ],
    [
        f_C_S.diff(C_A).subs({C_S: C_Ss, T: T_s}), 
        f_C_S.diff(C_S).subs({C_S: C_Ss, T: T_s}), 
        f_C_S.diff(T).subs({C_S: C_Ss, T: T_s}),
    ],
    [
        f_T.diff(C_A).subs({C_A: C_As, C_S: C_Ss, T: T_s, U: U_s, T_0: T_0s}), 
        f_T.diff(C_S).subs({C_A: C_As, C_S: C_Ss, T: T_s, U: U_s, T_0: T_0s}), 
        f_T.diff(T).subs({C_A: C_As, C_S: C_Ss, T: T_s, U: U_s, T_0: T_0s}),
    ],
])

b = sympy.matrices.Matrix([
    [f_C_A.diff(U).subs({C_A: C_As, T: T_s})],
    [f_C_S.diff(U).subs({C_S: C_Ss, T: T_s})],
    [f_T.diff(U).subs({C_A: C_As, C_S: C_Ss, T: T_s, U: U_s, T_0: T_0s})],

])

c = sympy.matrices.Matrix([
    [0, 0, 1],
])

d = 0

state_vector = sympy.matrices.Matrix([
    [C_A],
    [C_S],
    [T],
])

## Linear Transfer Function

$$G(s)=c\left(sI-A\right)^{-1}+d$$

In [10]:
# Trandfer function
(c * (s * sympy.matrices.eye(3) - A)**-1 * b + sympy.matrices.Matrix([d]))[0]

-3.67337620111097e-5*(1.0*s**2 + 1.58473743578246*s + 0.0951400801428692)/(1.0*s**3 + 1.38576553049263*s**2 + 0.170469031450751*s + 0.00548530996943723)

In [11]:
system_parameter_dict = {
    'parameters': {
        'k_10': 4e14,
        'E_1': 1.28e5,  
        'k_20': 1e84,
        'E_2': 8e5,
        'H_1': -45400,
        'H_2': -3.2e5,
        'rho': 12.4,
        'C_p': 254,
        'A': 5.3,
        'T_c': 373,
        'V': 4000,
    },
    'inlet': {
        'F_A0': 1075,
        'F_S0': 750,
        'F_B0': 1275,
        'F': F,
        'v_0': v_0,
        'C_A0': C_A0, 
        'C_B0': C_B0, 
        'C_S0': C_S0, 
    },
    'steady_state': {
        'T_0': T_0s,
        'U': U_s,
        'T': T_s,
        'C_A': C_As,
        'C_S': C_Ss,
    },
    'state_space': {
        'A': np.array(A, dtype=float).tolist(),
        'b': np.array(b, dtype=float).tolist(),
        'c': np.array(c, dtype=float).tolist(),
        'd': d,
    }
}

In [12]:
with open('system_parameter_dict.json', 'w') as _write_file:
    json.dump(system_parameter_dict, _write_file, indent=4)